<a href="https://colab.research.google.com/github/SyncHJ/ESG_MBA_data_analytics/blob/main/ESG_MBA_10%EC%9B%94_08%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 69.4 MB/s 


In [2]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.2 MB/s 


In [4]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import xlsxwriter
import urllib.request
from konlpy.tag import Okt
from matplotlib import font_manager, rc
from openpyxl import Workbook
from gensim.models.word2vec import Word2Vec
# import nltk
# from nltk.corpus import stopwords

font_path ='./NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
rc('font', family='AppleGothic')

okt = Okt()

# driver_path = r'/Users/mac_h/Desktop'
# os.chdir(driver_path)
data = pd.read_excel('/content/ESG_sp.xlsx',engine='openpyxl')
# csa=pd.read_csv("ESG_sp.csv", encoding='cp949')

In [5]:
print(len(data)) #데이터 값이 몇 개 있는지 확인

10438


In [7]:
import re
 
def preprocessing(text):
    # 특수문자나 이모티콘 등 아래의 특수기호들을 제거합니다(%등은 남김).
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   
    #  re.sub는 위에서 특수문자를 제거한 text에서 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

def clean_text(text):
    text = str(text)
    text = re.sub(r'\[.*?\]', '', text) # 대괄호 안에 텍스트 제거 
    text = re.sub(r'\(.*?\)', '', text) # 소괄호 안에 텍스트 제거
 
    return text

In [10]:
# 불용어 제거 작업
import json #json 파일을 불러오기 한 뒤에 아래 Stopworkd.extend에서 추가를 해서 지울 것들 만들면 됨
with open('korean_stopword.json',encoding='utf8') as f:  # 분석하고자 파일과 동일한 폴더(위치)에 있어야 한다
    stopwords = json.load(f)

In [11]:
cs = pd.DataFrame()
cs['keywords'] = data['keywords'] 
cs['text_clean']=pd.DataFrame(cs['keywords'].apply(lambda x: clean_text(x)))

# stopwords를 하기 위해서는 tokenize를 먼저 해야 함 (계속해서 불필요한 단어들을 보면서 제거)
cs['text_ko'] = [okt.morphs(doc) for doc in cs.text_clean]
cs['text_tokenized'] = cs.text_ko.apply(lambda x: [ a for a in x if a not in stopwords])

In [12]:
print(cs)

                                                keywords  \
0      SK케미칼,발간,TCFD,보고서,기후변화,공유,대응,전략,SK케미칼,생산,제품,생애...   
1      제주은행,사회,공헌비율,최고,은행,제주은행,시중,은행,이익,대비,사회,공헌,비율,국...   
2      개최,KICOX,국민,참여,혁신,아이디어,공모전,한국산업단지공단,국민,목소리,아이디...   
3      SK케미칼,기후,변화,대응,전략,TCFD,보고서,발간,SK,케미칼,TCFD,보고서,...   
4      삼성,LG전자,KES,참가,혁신제품,경쟁,삼성전자,스마트싱스,활용,체험존,마련,삼성...   
...                                                  ...   
10433  예산,심덕섭,고창,군수,취임,고창군,확보,취임식,민선,심덕섭,전북,고창,군수,고창,...   
10434  인천도시공사,수상,ESG,공공,기관,종합,대상,지방공기업,발행,ESG,채권,ESG,...   
10435  예방,대청호,녹조,처리,금강,유역,홍수,가뭄,예방,재해,주목적,준공,대청호,대전,충...   
10436  순천향대,지주,자회사,IR,대회,Value-UP,R&BD,세미나,개최,아산,순천향대...   
10437  국립,생태원,한국가스공사,업무협약,멸종,위기,보전,업무,협약,서천,국립생태원,원장,...   

                                              text_clean  \
0      SK케미칼 발간 TCFD 보고서 기후변화 공유 대응 전략 SK케미칼 생산 제품 생애...   
1      제주은행 사회 공헌비율 최고 은행 제주은행 시중 은행 이익 대비 사회 공헌 비율 국...   
2      개최 KICOX 국민 참여 혁신 아이디어 공모전 한국산업단지공단 국민 목소리 아이디...   
3      SK케미칼 기후 변화 대응 전략 TCFD 보고서 발간 SK

In [13]:
tokenized_doc=cs['text_tokenized']

## Word2Vec

In [14]:
tokenized_do= list(tokenized_doc)

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

tokenizer.fit_on_texts(tokenized_do)

In [15]:
# 정수 인코딩
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_do)

vocab_size = len(word2idx) + 1 
print('단어 집합의 크기 :', vocab_size)

단어 집합의 크기 : 37932


In [16]:
import gensim
from gensim.models import Word2Vec, FastText

model = Word2Vec(tokenized_do,  min_count=3)

In [17]:
model.wv.vectors.shape 

(21848, 100)

In [19]:
from gensim.models import KeyedVectors
import os

model.wv.save_word2vec_format('esg_mba') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('esg_mba')
!python -m gensim.scripts.word2vec2tensor --input esg_mba --output esg_mba

2022-10-07 05:52:05,049 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py --input esg_mba --output esg_mba
2022-10-07 05:52:05,049 - utils_any2vec - INFO - loading projection weights from esg_mba
2022-10-07 05:52:06,196 - utils_any2vec - INFO - loaded (21848, 100) matrix from esg_mba
2022-10-07 05:52:07,770 - word2vec2tensor - INFO - 2D tensor file saved to esg_mba_tensor.tsv
2022-10-07 05:52:07,770 - word2vec2tensor - INFO - Tensor metadata file saved to esg_mba_metadata.tsv
2022-10-07 05:52:07,773 - word2vec2tensor - INFO - finished running word2vec2tensor.py
